In [2]:
import sys
import os
import numpy as np
import pandas as pd
import urllib
import subprocess
import re
import tempfile
import torch
from typing import List
# import qgrid

In [3]:
BASE_DIR = os.path.abspath(os.getcwd()+'/../..')  # /home/gil/dev/NEBULA2/
os.chdir(os.getcwd()+'/../..')

In [3]:
from nebula_api.nebula_enrichment_api import *
from experts.common.RemoteAPIUtility import RemoteAPIUtility
from nebula_api.mdmmt_api import mdmmt_api

In [4]:
nre = NRE_API()
api = RemoteAPIUtility()
mdmmt = mdmmt_api.MDMMT_API()

INFO:tensorflow:Restoring parameters from /home/gil/dev/NEBULA2/nebula_api/mdmmt_api/ckpts/vggish_model.ckpt


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.

In [5]:
movies = ['Movies/114206952',
'Movies/114207205',
'Movies/114207398',
'Movies/114207499',
'Movies/114207361',
'Movies/114207740',
'Movies/114207908',
'Movies/114208744',
'Movies/114206724',
'Movies/114206548',
'Movies/114206264']

In [6]:
from IPython.display import Javascript
from IPython.display import HTML, display
import base64


def download_video_file(movie, fname='/tmp/video_file.mp4'):    
    if os.path.exists(fname):
        os.remove(fname)
    query = 'FOR doc IN Movies FILTER doc._id == "{}" RETURN doc'.format(movie)
    cursor = api.db.aql.execute(query)
    url_prefix = "http://ec2-3-120-189-231.eu-central-1.compute.amazonaws.com:7000/"
    url_link = ''
    for doc in cursor:
        url_link = url_prefix+doc['url_path']
        url_link = url_link.replace(".avi", ".mp4")   
        print(url_link)
        urllib.request.urlretrieve(url_link, fname) 
    return fname
    # video = cv2.VideoCapture(self.temp_file)
    # fps = video.get(cv2.CAP_PROP_FPS)
    # return(fps, url_link)



def read_video_segm(abspath, t_beg, t_end):
    cmd = f'ffmpeg -y -ss {t_beg} -i {abspath} -max_muxing_queue_size 9999  -loglevel error -f mp4 -vf scale="(floor(112/ih * iw/2))*2:112"  -c:a copy  -movflags frag_keyframe+empty_moov -t {t_end - t_beg} pipe:1 -nostats -hide_banner -nostdin'
    p = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE)
    assert p.returncode == 0, cmd
    buf = p.stdout
    return buf

video_id_cnt = 0    
class VideoElem:
    def __init__(self, fname, t_start=0, t_end=999):
        with open(fname, 'rb') as f:
            #data = base64.standard_b64encode(f.read())
            buf = read_video_segm(fname, t_start, t_end)
            data = base64.standard_b64encode(buf)
        global video_id_cnt
        video_id_cnt += 1
        self.video_id_cnt = video_id_cnt
        elem = HTML(f"""
            <video id="video_{self.video_id_cnt}" autoplay loop muted>
                <source src="data:video/mp4;base64,{data.decode('ascii')}" type="video/mp4">
            </video>        
        """)
        display(elem)
    
    def hide(self):
        js = f'$("#video_{self.video_id_cnt}").hide()'
        display(Javascript(js))
        
    def show(self):
        js = f'$("#video_{self.video_id_cnt}").show()'
        display(Javascript(js))

    def remove(self):
        js = f'$("#video_{self.video_id_cnt}").remove()'
        display(Javascript(js))
        
def mdmmt_video_encode(start_f, stop_f, path='/tmp/video_file.mp4', freq=24):
        t_start = start_f//freq
        t_end = stop_f//freq
        if t_start == t_end:
            t_start = t_start - 1
        print("Start/stop", t_start, " ", t_end)
        if (t_end - t_start) >= 1:
            vemb = mdmmt.encode_video(
                mdmmt.vggish_model,  # adio modality
                mdmmt.vmz_model,  # video modality
                mdmmt.clip_model,  # image modality
                mdmmt.model_vid,  # aggregator
                path, t_start, t_end)
            return(vemb)
        else:
            print("Stage too short")
            return(None)

In [7]:
def range_get_val_for_frame(frame,val,data):
    return [x[val] for x in data if x['start']<=frame and x['stop']>frame]
def tracker_get_box_for_frame(frame,data):
    return [x['bboxes'][str(frame)] for x in data if x['start']<=frame and x['stop']>frame]
def step_get_val_for_frame(frame, val,data):
    return [x[val] for x in data if int(x['frame_id'])==frame]

In [8]:
def normalize_movie(mid):
    action_data = nre.get_all_expert_data("Actions", mid)
    object_data = nre.get_all_expert_data("Object", mid)
    clip_data = nre.get_clip_data(mid)
    vcomet_data = nre.get_vcomet_data(mid)
    movie_info = api.get_movie_info(mid)
    frame_array = []
    for i in range(movie_info['last frame']):
        scene_element_val = range_get_val_for_frame(i,'scene_element', object_data)        
        frame_data = {'arango_id': mid, 'frame_id': i, 
                      'step_actor_id': step_get_val_for_frame(i, 'actor_id', action_data),
                      'step_description': step_get_val_for_frame(i, 'description', action_data),
                      'step_box': step_get_val_for_frame(i,'box', action_data),
                      'tracker_description': range_get_val_for_frame(i, 'description', object_data),
                      'scene_element': scene_element_val[0] if scene_element_val else None,
                      'tracker_box': tracker_get_box_for_frame(i, object_data),
                      'simulated_expert': []}                          
        scene = frame_data['scene_element']
        # Verify with dima what is order of scene in clip vs vcomet
        # frame_data['clip_text'] = None
        # frame_data['clip_triplets'] = None
        frame_data['vcomet_places'] = None
        frame_data['vcomet_events'] = None
        frame_data['vcomet_actions'] = None
        if scene is not None:
            # frame_data['clip_text'] = clip_data[scene][0]
            # frame_data['clip_triplets'] = clip_data[scene][1]
            frame_data['vcomet_places'] = range_get_val_for_frame(i,'places',vcomet_data)
            frame_data['vcomet_events'] = range_get_val_for_frame(i,'events',vcomet_data)
            frame_data['vcomet_actions'] = range_get_val_for_frame(i,'actions',vcomet_data)                    
        frame_array.append(frame_data)
    return frame_array                                         

In [9]:
def create_normalized_frame(frame, experiment='default'):
    frame['experiment'] = experiment
    update_vars = ', '.join((['{}: @{}'.format(k,k) for k in frame.keys()]))
    query = 'UPSERT { arango_id: @arango_id, \
                                frame_id: @frame_id, experiment: @experiment}\
            INSERT  \
                {'+update_vars+', updates: 1}\
            UPDATE \
                { updates: OLD.updates + 1, '+update_vars+'} IN normalized_frames \
                    RETURN { doc: NEW, type: OLD ? \'update\' : \'insert\' }'       
    return api.db.aql.execute(query, bind_vars=frame)

def save_normalized_collection(frames, **kwargs):
    for frame in frames:
        try:
            create_normalized_frame(frame, **kwargs)
        except:
            print('Failed on {}'.format(frame['frame_id']))

def recreate_normalized_movie(mid, **kwargs):
    frames = normalize_movie(mid)
    save_normalized_collection(frames, **kwargs)

In [10]:
def get_normalized_frame(mid, frame, experiment='default'):
    query = "FOR doc IN normalized_frames FILTER doc.arango_id == '{}' AND doc.frame_id == {} AND doc.experiment == '{}' RETURN doc".format(mid,frame,experiment)
    print(query)
    cur = list(api.db.aql.execute(query))
    if cur:
        return cur[0]    # We only have one normalized frame (?)

def get_normalized_range(mid, start, stop, experiment='default'):
    query = "FOR doc IN normalized_frames FILTER doc.arango_id == '{}' AND doc.frame_id >= {} AND doc.frame_id < {} AND doc.experiment == '{}' RETURN doc".format(mid,start,stop,experiment)
    return list(api.db.aql.execute(query))    

def get_normalized_scene(mid, scene, experiment='default'):
    query = "FOR doc IN normalized_frames FILTER doc.arango_id == '{}' AND doc.scene_element == {} AND doc.experiment == '{}' RETURN doc".format(mid,scene,experiment)
    return list(api.db.aql.execute(query))        
def get_normalized_movie(mid, experiment='default'):
    query = "FOR doc IN normalized_frames FILTER doc.arango_id == '{}' AND doc.experiment == '{}' RETURN doc".format(mid,experiment)
    return list(api.db.aql.execute(query))
def get_or_create_normalized_video(mid, **kwargs):
    frames = get_normalized_movie(mid,**kwargs)
    if not frames:
        recreate_normalized_movie(mid, **kwargs)
        frames = get_normalized_movie(mid, **kwargs) 
    return pd.DataFrame(frames).drop(['_key','_id','_rev', 'updates'],axis=1).replace({np.nan: None})

In [48]:
def frame_to_concepts(frame)-> List:
    def transform_concept(c):
        exp = re.compile(r"^([a-zA-z]+)(\d*)$")
        r = exp.match(c)
        return r.group(1) if r else c
        
    pre_concepts = set(frame['tracker_description']).union(set(frame['step_description'])).union(set(frame['simulated_expert']))
    concepts = list(set(map(transform_concept,pre_concepts)))
    return concepts

def kgbart_fusion(frames) -> (List[str], List[str]):
    h, outname = tempfile.mkstemp(text=True)
    os.close(h)
    h, fname = tempfile.mkstemp(text=True)
    os.close(h)
    KGBART_MAIN = BASE_DIR+'/kgbart/KGBART/KGBART_training/decode_seq2seq.py'
    KGBART_CC_DIR = BASE_DIR+'/kgbart/downloaded/commongen_dataset'
    KGBART_MODEL_DIR = BASE_DIR+'/kgbart/output/best_model/model.best.bin'
    options = {
        'data_dir': KGBART_CC_DIR,
        'output_dir': os.path.dirname(outname),
        'input_file': fname,
        'model_recover_path': KGBART_MODEL_DIR,
        'output_file': os.path.basename(outname),
        'split': 'dev',
        'beam_size': 5,
        'forbid_duplicate_ngrams': True
    }
    all_concepts = []
    with open(fname, 'w') as f:
        for frame in frames:
            concepts = frame_to_concepts(frame)
            all_concepts.append(', '.join(concepts))
            f.write(' '.join(concepts)+'\n')
        
    # write expert tokens to input file
    
    cmdline = 'python '+KGBART_MAIN+' '+ ' '.join(['--{} {}'.format(k,v) for (k,v) in options.items()]) + '>/dev/null 2>&1'
    os.system(cmdline)
    with open(outname,'r') as f:
        rc = f.readlines()
    os.unlink(outname)
    os.unlink(fname)
    return all_concepts, rc

------------------------------------------------------Here be Scratchpad------------------------------------------------------

In [ ]:
mid = movies[2]
experiment = 'default'

In [15]:
video = VideoElem(download_video_file(mid))

http://ec2-3-120-189-231.eu-central-1.compute.amazonaws.com:7000/static/development/0017_Pianist_00_34_12_556-00_34_15_845.mp4


In [16]:
df = get_or_create_normalized_video(mid)
movie_info = api.get_movie_info(mid)

In [39]:
frames = df.iloc[movie_info['mdfs'][0]].to_dict('records')

In [80]:
frames[0]['step_description']=['talk', 'meet']
frames[0]['tracker_description'] = ['woman', 'man', 'clothing', 'car']

In [81]:
rc = kgbart_fusion(frames)

In [82]:
rc[0]

['man, clothing, meet, talk, woman, car',
 'carry/hold (an object), stand, Clothing, Man, listen to (a person), watch (a person)',
 'carry/hold (an object), Human face12, Clothing, stand, Man, listen to (a person), watch (a person)']

In [83]:
rc[1]

['A man meets a woman in a car and they talk about clothes.\n',
 'a person listens to how a person is carrying an object and holds it as he or she listens to someone else (a person) who is standing next to him or her.\n',
 'a person listens to how a person is carrying an object and holds it in his or her hands as he or she listens to the comments of another person or person standing nearby and faces him or her.\n']

In [ ]:
ignore_columns = ['step_actor_id', 'step_box', 'tracker_box']
pdf = df.drop(ignore_columns,axis=1).set_index('frame_id',append=True).reset_index()

In [ ]:
movie_info['mdfs']

In [ ]:
emb_image = mdmmt_video_encode(*movie_info['scene_elements'][0])

In [ ]:
emb_text = mdmmt.encode_text("nazi soldiers threatening jewish immigrants")

In [ ]:
torch.matmul(emb_text,emb_image)

In [ ]:
pdf.columns

In [ ]:
df.at[30,'simulated_expert'] = ['gil', 'dan']

In [ ]:
save_normalized_collection(df.to_dict('records'),experiment='giltest')

In [ ]:
df1 = get_or_create_normalized_video(mid,experiment='giltest')

In [ ]:
api.get_expert_status("Actions",'movies')

In [ ]:
api.db.create_collection("giltest")

In [ ]:
nre.force_start_expert("Actions")

In [ ]:
new_movies = api.get_new_movies()
for x in new_movies[1:]:
    api.change_status_movie('updated', x)

In [ ]:
api.get_new_movies()